In [2]:
from huggingface_hub import login
login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import torch
device = "cuda" # the device to load the model onto

model_name = "Qwen/Qwen1.5-MoE-A2.7B"

# 4-bit QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                 # QLoRA = 4-bit base model
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",         # QLoRA uses nf4
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

ERROR! Session/line number was not unique in database. History logging moved to new session 5


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

model-00008-of-00008.safetensors:   0%|          | 0.00/668M [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

In [4]:
model.eval()
inputs = tokenizer("Hello!", return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(out[0], skip_special_tokens=True))

Hello! I'm a 20 year old girl from the Netherlands. I'm a student and I'm


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
from peft import IA3Config, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

config = LoraConfig(r = 4,
                    lora_alpha=4,
                    target_modules = ["gate", "q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
                    lora_dropout=0.1
                    )

lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

trainable params: 63,706,752 || all params: 14,379,490,944 || trainable%: 0.4430


In [6]:
from datasets import load_dataset

# dataset = load_dataset("Na0s/sft-ready-Text-Generation-Augmented-Data", split="train")

In [7]:
# print(dataset)

In [8]:
# def tokenize(batch):
#     texts = []
#     for prompt, completion in zip(batch["prompt"], batch["completion"]):
#         text = tokenizer.apply_chat_template(
#             [
#                 {"role": "user", "content": prompt},
#                 {"role": "assistant", "content": completion},
#             ],
#             tokenize=False,
#             add_generation_prompt=False,
#         )

#         if isinstance(text, list):
#             text = "".join(text)

#         texts.append(text)

#     return tokenizer(
#         texts,
#         truncation=True,
#         padding=False,
#         max_length=2048
#     )

# batch = dataset.select(range(3))
# tokens = tokenize(batch)
# tokenizer.decode(tokens["input_ids"][0])


In [9]:
# tokenized_dataset = dataset.map(
#     tokenize,
#     batched=True,
#     num_proc=75,
#     remove_columns=["prompt", "completion"],
# )

# print(tokenized_dataset[0])

In [10]:
tokenized_dataset = load_dataset("kaaiiii/tokenized_data_for_MoE", split="train")

README.md:   0%|          | 0.00/331 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

data/train-00000-of-00026.parquet:   0%|          | 0.00/212M [00:00<?, ?B/s]

data/train-00001-of-00026.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

...

data/train-00023-of-00026.parquet:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

data/train-00024-of-00026.parquet:   0%|          | 0.00/46.6M [00:00<?, ?B/s]

data/train-00025-of-00026.parquet:   0%|          | 0.00/64.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7667416 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [11]:
# print(tokenized_dataset[0:5])
# tokenized_dataset.save_to_disk("./tokenized_dataset")

In [13]:

tokenizer.decode(tokenized_dataset["input_ids"][0])

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n0.002 = 1000 \n1 = x?<|im_end|>\n<|im_start|>assistant\nTo find the value of x, we can set up a proportion using the given information:\n\n0.002/1000 = 1/x\n\nTo solve for x, we can cross multiply:\n\n0.002 * x = 1000 * 1\n\n0.002x = 1000\n\nDividing both sides by 0.002:\n\nx = 1000 / 0.002\n\nx = 500,000\n\nTherefore, 1 is equal to 500,000 in this proportion.<|im_end|>\n'

In [14]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print(lora_model.hf_device_map)

CUDA available: True
Device count: 1
Current device: 0
{'': 0}


In [15]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [17]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # causal LM
)

training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    max_steps=500,               # try a small number first
    learning_rate=1e-5,
    weight_decay=0.03,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    bf16=True,

    logging_steps=1,
    logging_strategy="steps",
    logging_first_step=True,
    report_to="none",      # prevents WandB from hijacking logs
    disable_tqdm=False,    # ensure progress bar is visible

    save_strategy="steps",
    save_steps=200,
    save_total_limit=10,
    save_safetensors=True,
)

trainer = Trainer(
    model=lora_model,          # your LoRA-wrapped model
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()

trainer.model.save_pretrained("Qwen1.5_MoE_lora_model_500steps")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/workspace/MoE_pruning/venv/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.002200
2,1.633600


KeyboardInterrupt: 

In [ ]:
trainer.model.push_to_hub("kaaiiii/Qwen1.5b_MoE_LoRA")

In [ ]:
# from trl import SFTConfig
# import logging
# logging.basicConfig(level=logging.INFO)

# from transformers import logging as hf_logging
# hf_logging.set_verbosity_info()

# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-MoE-A2.7B", use_fast=True)
# print(lora_model.device)

# trainer = SFTTrainer(
#     model = lora_model,
#     train_dataset = tokenized_dataset["train"],
#     # processing_class = tokenizer,
#     args = SFTConfig(
#         dataset_kwargs={"skip_prepare_dataset": True},
#         per_device_train_batch_size = 8,
#         gradient_accumulation_steps = 1,
#         packing = True,
#         group_by_length = True,
#         warmup_steps = 5,
#         bf16 = True,
#         max_steps=500,
#         learning_rate = 1e-4,
#         optim = "adamw_8bit",
#         weight_decay = 0.03,
#         lr_scheduler_type = "cosine",
#         seed = 3407,
#         eval_strategy="no",
#         do_eval=False,
#         output_dir = "./outputs",
#         push_to_hub=True,
#         remove_unused_columns=False,
#         logging_steps = 5,
#         disable_tqdm=False,
#         report_to="none"
#     )
# )

# torch.cuda.empty_cache()

# trainer.train()

# trainer.model.save_pretrained("Qwen1.5_MoE_lora_model")